# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [1]:
import pandas as pd

RANDOM_STATE = 110

train_data = pd.read_csv("../../../data/train_data_0816.csv")
test_data = pd.read_csv("../../../data/test_data_0816.csv")

In [2]:
# 공통 변수 리스트
com_variables_train = [
    'target', 'Model.Suffix', 'Workorder', 'WorkMode Collect Result'
    , 'Dispenser_1', 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

com_variables_test = [
    'target', 'Set ID', 'Model.Suffix', 'Workorder'
    , 'WorkMode Collect Result', 'Dispenser_1'
    , 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

In [3]:
# 공정 이름 필터링 후 공통 변수와 결합
def create_dataset(train_data, test_data, process_name, com_variables_train, com_variables_test):
    # 열 이름 필터링
    Process_Desc_col = train_data.filter(like=process_name).columns
    
    # train 데이터셋 생성
    final_columns_train = list(Process_Desc_col) + com_variables_train
    train_dataset = train_data[final_columns_train]
    
    # test 데이터셋 생성
    final_columns_test = list(Process_Desc_col) + com_variables_test
    test_dataset = test_data[final_columns_test]
    
    return train_dataset, test_dataset

# 공통 변수 정의
## com_variables_train = [...]  -> 이전 코드에서 정의한 변수 사용
## com_variables_test = [...]   -> 이전 코드에서 정의한 변수 사용

# 데이터셋 생성
train_data_dam, test_data_dam = create_dataset(train_data, test_data, '_Dam', com_variables_train, com_variables_test)
train_data_fill1, test_data_fill1 = create_dataset(train_data, test_data, '_Fill1', com_variables_train, com_variables_test)
train_data_fill2, test_data_fill2 = create_dataset(train_data, test_data, '_Fill2', com_variables_train, com_variables_test)
train_data_autoclave, test_data_autoclave = create_dataset(train_data, test_data, '_AutoClave', com_variables_train, com_variables_test)

---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [4]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# 'Normal'과 'AbNormal'을 숫자로 변환
train_data_fill1['target'] = train_data_fill1['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3

def objectiveCatBoost(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'iterations': trial.suggest_int('iterations', 400, 1500),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.4),
        'depth': trial.suggest_int('depth', 3, 14),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.05, 5.0, log=True),
        'random_strength': trial.suggest_float('random_strength', 0.1, 10.0),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 10.0),
        'border_count': trial.suggest_int('border_count', 70, 270),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.5, 1.2),

        'random_seed': RANDOM_STATE,
        'eval_metric': 'F1',
        'logging_level': 'Silent',
        'boosting_type': 'Plain'
    }

    model = CatBoostClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_fill1.drop("target", axis=1),
    train_data_fill1["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveCatBoost(trial, x_train, y_train, x_val, y_val), n_trials=200)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
[I 2024-08-18 23:09:17,672] A new study created in memory with name: no-name-0fd6e62b-2145-460e-a6f6-949bf3c391ca
[I 2024-08-18 23:09:29,436] Trial 0 finished with value: 0.22668579626972737 and parameters: {'iterations': 527, 'learning_rate': 0.2804593836111537, 'depth': 7, 'l2_leaf_reg': 0.8404950643051162, 'random_strength': 6.826956469952126, 'bagging_temperature': 6.85638497930364, 'border_count': 261, 'scale_pos_weight': 0.9460560575741956}. Best is trial 0 with value: 0.22668579626972737.
[I 2024-08-18 23:09:50,763] Trial 1 finished with value: 0.2139384116693679 and parameters: {'iterations': 964, 

Best trial: score 0.24999999999999994, 
params {'iterations': 1080, 'learning_rate': 0.27700365060588783, 'depth': 5, 'l2_leaf_reg': 1.253973786965849, 'random_strength': 9.22514112631014, 'bagging_temperature': 8.810531538725536, 'border_count': 205, 'scale_pos_weight': 1.1080635788286048}


Best trial: score 0.24999999999999994,  
params {'iterations': 1080, 'learning_rate': 0.27700365060588783, 'depth': 5, 'l2_leaf_reg': 1.253973786965849,  
'random_strength': 9.22514112631014, 'bagging_temperature': 8.810531538725536, 'border_count': 205, 'scale_pos_weight': 1.1080635788286048}

.